# Install required libraries in Colab

In [ ]:
!pip install gdown datasets --quiet

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import zipfile
import gdown
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import json
import requests
from google.colab import files

# 1. Download 11k Hands dataset

In [ ]:
url_11k_images = "https://drive.google.com/uc?id=1KcMYcNJgtK1zZvfl_9sTqnyBUTri2aP2"
output_11k_images = "11k_hands.zip"
gdown.download(url_11k_images, output_11k_images, quiet=False)

In [ ]:
extract_dir_11k = "./11k_hands"
with zipfile.ZipFile(output_11k_images, 'r') as zip_ref:
    zip_ref.extractall(extract_dir_11k)
print(f"Extracted 11k Hands images to {extract_dir_11k}")
os.remove(output_11k_images)

In [ ]:
url_11k_csv = "https://drive.google.com/uc?id=1RC86-rVOR8c93XAfM9b9R45L7C2B0FdA"
output_11k_csv = "./11k_hands/HandInfo.csv"
gdown.download(url_11k_csv, output_11k_csv, quiet=False)

In [ ]:
# Load Hugging Face token from environment variable
hf_token = os.getenv('HUGGINGFACE_TOKEN')
if not hf_token:
    print("No HUGGINGFACE_TOKEN environment variable found. Skipping HaGRID test data.")

In [ ]:
ann_test_path = "ann_test.json"
if hf_token:
    headers = {"Authorization": f"Bearer {hf_token}"}
    ann_test_url = "https://huggingface.co/datasets/abhishek/hagrid/raw/main/ann_test.json"
    response = requests.get(ann_test_url, headers=headers)
    if response.status_code == 200:
        # Debug: Print the first 500 characters of the response to check content
        print("Debug: First 500 characters of ann_test.json response:")
        print(response.text[:500])
        if response.text.strip():
            with open(ann_test_path, "w") as f:
                f.write(response.text)
            print("Downloaded ann_test.json successfully.")
        else:
            print("Downloaded ann_test.json is empty. Proceeding without HaGRID test data.")
            ann_test_path = None
    else:
        print(f"Failed to download ann_test.json (status: {response.status_code}). Proceeding without HaGRID test data.")
        ann_test_path = None
else:
    print("No token provided. Skipping HaGRID test data.")
    ann_test_path = None

In [ ]:
class GestureDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        image = torch.tensor(image.transpose((2, 0, 1)), dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# 4. Define paths and gesture classes
DATASETS = {
    "11k_hands": "./11k_hands/Hands/",
    "hagrid_test": "./hagrid_test/"  # Placeholder for manual upload
}

GESTURE_CLASSES = {
    0: "point_right",  # 👉 Move to next slide
    1: "point_left",   # 👈 Move to previous slide
    2: "pointer",      # ☝️ Display pointer
    3: "palm_out",     # 🤚 Erase drawing
    4: "stop",         # ✋ Stop presentation
    5: "open_hand",    # 🖐️ First slide
    6: "peace",        # ✌️ Last slide
    7: "draw",         # ✍️ Draw on screen
    8: "save",         # 💾 Save slide/drawing
    9: "highlight"     # 🖌️ Highlight text
}

hagrid_label_map = {
    "point": 0,       # point_right
    "palm": 4,        # stop
    "peace": 6,       # peace
    "fist": 8,        # save
    "call": 3         # palm_out
}